In [1]:
import cv2 
import numpy as np
from PIL import Image
def createdepthmap(left,right,img,lrsimilarity,tbsimilarity,fimage):
    depth_map = cv2.normalize(src=fimage, dst=fimage, beta=0, alpha=255, norm_type=cv2.NORM_MINMAX);
    depth_map = np.uint8(depth_map)
    if (tbsimilarity > lrsimilarity):
        depth_map = cv2.bitwise_not(depth_map)
    depth_image = Image.fromarray(depth_map, mode="L")
    colours_array  = np.array(left.resize(img.size)
                                .rotate(-90, expand=True)
                                .getdata()
                    ).reshape(img.size + (3,))
    return depth_image,colours_array

In [2]:
import cv2
import numpy as np
def lrmatcher(window_size,limage,rimage):
    lmatcher = cv2.StereoSGBM_create(
                minDisparity=0,
                numDisparities=16,
                blockSize=5,
                P1=8 * 3 * window_size ** 2,
                P2=32 * 3 * window_size ** 2,
            )         
    rmatcher = cv2.ximgproc.createRightMatcher(lmatcher)
    wls_filter = cv2.ximgproc.createDisparityWLSFilter(matcher_left=lmatcher)
    wls_filter.setLambda(80000)
    wls_filter.setSigmaColor(1.2)
    left_disparity  = lmatcher.compute(limage, rimage)
    right_disparity = rmatcher.compute(rimage, limage)
    left_disparity  = np.int16(left_disparity)
    right_disparity = np.int16(right_disparity)
    imagefiltered  = wls_filter.filter(left_disparity, limage, None, right_disparity)
    return imagefiltered

In [4]:
pip install pyntcloud

     -------------------------------------- 346.3/346.3 KB 1.0 MB/s eta 0:00:00



You should consider upgrading via the 'c:\users\mookambika\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [4]:
import pandas as pd
import numpy as np
import cv2
import argparse
from pyntcloud import PyntCloud   
from PIL import Image
def createmesh(filename):
    img = Image.open(filename).convert('RGB')
    w, h = img.size
    r  = img.crop( (0,0, w/2, h))
    l   = img.crop( (w/2, 0, w,h))
    t    = img.crop( (0,0, w, h/2))
    b = img.crop( (0, h/2, w,h))
    try:
        l_r_same = mse(np.array(r),np.array(l))
        t_b_same = mse(np.array(t),np.array(b))
    except:
        return 0
    if (t_b_same < l_r_same):
        l  = b
        r = t
    image_l  = np.array(l) 
    image_r = np.array(r) 
    window_size = 15
    filtered_image  =lrmatcher(window_size,image_l,image_r)
    depth_image,coloursarray=createdepthmap(l,r,img,l_r_same,t_b_same,filtered_image)
    indicesarray = np.moveaxis(np.indices(img.size), 0, 2)
    image_Array    = np.dstack((indicesarray, coloursarray)).reshape((-1,5))
    df = pd.DataFrame(image_Array, columns=["x", "y", "red","green","blue"])
    depths_array = np.array(depth_image.resize(img.size)
                                        .rotate(-90, expand=True)
                                        .getdata())     
    df.insert(loc=2, column='z', value=depths_array)
    df[['red','green','blue']] = df[['red','green','blue']].astype(np.uint)
    df[['x','y','z']] = df[['x','y','z']].astype(float)
    df['z'] = df['z']*5
    cloud = PyntCloud(df)
    cloud.to_file(filename+".ply", also_save=["mesh","points"],as_text=True)
    return 1
def mse(imageA, imageB):
    err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
    err /= float(imageA.shape[0] * imageA.shape[1])
    return err

In [10]:
createmesh('home.png')

1

In [18]:
print(cv2.__version__)

4.5.3
